### Installazione e Importazione delle Librerie
In questa sezione, installeremo e importeremo tutte le librerie necessarie per la segmentazione delle immagini. Le librerie includono strumenti per la manipolazione dei dati, la costruzione e l'addestramento del modello, e l'analisi dei risultati.


In [ ]:
!pip install -U torch torchvision matplotlib pillow tqdm opencv-python albumentations torchsummary segmentation-models-pytorch

In [ ]:
# Importare le librerie necessarie
import os
import copy
import torch
import torchvision.transforms as transforms
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

from torchvision import models, transforms as T
import torchvision
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.autograd import Variable
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2
import time
from torchsummary import summary

!pip install -q segmentation-models-pytorch
import segmentation_models_pytorch as smp

from torchvision.datasets import VOCSegmentation
from torchvision.models.segmentation import deeplabv3_resnet101


### Preprocessamento delle Immagini
Questa sezione descrive il preprocessamento delle immagini, che include il ridimensionamento, la normalizzazione e la suddivisione del dataset in set di addestramento e di test. Questi passaggi sono cruciali per garantire che i dati siano in un formato adatto per l'addestramento del modello.


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")

# Definizione dei percorsi dei dati
#IMAGE_PATH = 'immagini/images/validation/'
#MASK_PATH = 'immagini/annotations/validation/'
IMAGE_PATH = 'immagini3/images/'
MASK_PATH = 'immagini3/annotations/'
IMAGE_PATH2 = 'immagini5/images/'
MASK_PATH2 = 'immagini5/annotations/'

risoluz_vert = 192
risoluz_orizz = 384


n_classes = 30  #150 Numero delle classi nel dataset ADEChallengeData2016


In [ ]:
def create_df():
    name = []
    for dirname, _, filenames in os.walk(IMAGE_PATH):
        for filename in filenames:
            name.append(filename.split('.')[0])
    
    return pd.DataFrame({'id': name}, index = np.arange(0, len(name)))

df = create_df()
print('Total Images: ', len(df))

In [ ]:
#split data
X_trainval, X_test = train_test_split(df['id'].values, test_size=0.1, random_state=19)
X_train, X_val = train_test_split(X_trainval, test_size=0.15, random_state=19)

print('Train Size   : ', len(X_train))
print('Val Size     : ', len(X_val))
print('Test Size    : ', len(X_test))

In [ ]:
img = Image.open(IMAGE_PATH + df['id'][100] + '.jpg')
mask = Image.open(MASK_PATH + df['id'][100] + '.png')
print('Image Size', np.asarray(img).shape)
print('Mask Size', np.asarray(mask).shape)


plt.imshow(img)
plt.imshow(mask, alpha=0.6)
plt.title('Picture with Mask Applied')
plt.show()

In [ ]:
class DriveDataset(Dataset):
    
    def __init__(self, img_path, mask_path, X, mean, std, transform=None, patch=False):
        self.img_path = img_path
        self.mask_path = mask_path
        self.X = X
        self.transform = transform
        self.patches = patch
        self.mean = mean
        self.std = std
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        img = cv2.imread(self.img_path + self.X[idx] + '.jpg')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.mask_path + self.X[idx] + '.png', cv2.IMREAD_GRAYSCALE)
        
        if self.transform is not None:
            aug = self.transform(image=img, mask=mask)
            img = Image.fromarray(aug['image'])
            mask = aug['mask']
        
        if self.transform is None:
            img = Image.fromarray(img)
        
        t = T.Compose([T.ToTensor(), T.Normalize(self.mean, self.std)])
        img = t(img)
        mask = torch.from_numpy(mask).long()
        
        if self.patches:
            img, mask = self.tiles(img, mask)
            
        return img, mask
    
    def tiles(self, img, mask):

        img_patches = img.unfold(1, 512, 512).unfold(2, 768, 768) 
        img_patches  = img_patches.contiguous().view(3,-1, 512, 768) 
        img_patches = img_patches.permute(1,0,2,3)
        
        mask_patches = mask.unfold(0, 512, 512).unfold(1, 768, 768)
        mask_patches = mask_patches.contiguous().view(-1, 512, 768)
        
        return img_patches, mask_patches

In [ ]:
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

t_train = A.Compose([A.Resize(risoluz_vert, risoluz_orizz, interpolation=cv2.INTER_NEAREST), A.HorizontalFlip(), A.VerticalFlip(), 
                     A.GridDistortion(p=0.2), A.RandomBrightnessContrast((0,0.5),(0,0.5)),
                     A.GaussNoise()])

t_val = A.Compose([A.Resize(risoluz_vert, risoluz_orizz, interpolation=cv2.INTER_NEAREST), A.HorizontalFlip(),
                   A.GridDistortion(p=0.2)])

#datasets
train_set = DriveDataset(IMAGE_PATH, MASK_PATH, X_train, mean, std, t_train, patch=False)
val_set = DriveDataset(IMAGE_PATH, MASK_PATH, X_val, mean, std, t_val, patch=False)

#dataloader
batch_size= 3 

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=True)

### Definizione del Modello
In questa parte, definiremo l'architettura del modello di deep learning utilizzato per la segmentazione delle immagini. Discuteremo delle scelte di architettura e delle funzioni di perdita utilizzate per l'addestramento.


In [ ]:
#model = smp.Unet('mobilenet_v2', encoder_weights='imagenet', classes=n_classes, activation=None, encoder_depth=5, decoder_channels=[256, 128, 64, 32, 16])

In [ ]:
#model = smp.DeepLabV3Plus('resnet50', encoder_weights='imagenet', classes=n_classes, activation=None)

In [ ]:
#model = smp.FPN('efficientnet-b0', encoder_weights='imagenet', classes=n_classes, activation=None)

In [ ]:
#model = smp.Unet('resnet34', encoder_weights='imagenet', classes=n_classes, activation=None)

In [ ]:
model_name = 'UNet-ResNet'

# Carica il modello
model = torch.load(model_name + '.pt')

# Metti il modello in modalità di valutazione
model.eval()

### Training del Modello
Qui dettagliamo il processo di addestramento del modello, inclusi i parametri di addestramento, le tecniche di ottimizzazione, e il monitoraggio delle metriche di prestazione durante l'addestramento.


In [ ]:
def pixel_accuracy(output, mask):
    with torch.no_grad():
        output = torch.argmax(F.softmax(output, dim=1), dim=1)
        correct = torch.eq(output, mask).int()
        accuracy = float(correct.sum()) / float(correct.numel())
    return accuracy

In [ ]:
def mIoU(pred_mask, mask, smooth=1e-10, n_classes=n_classes):
    with torch.no_grad():
        pred_mask = F.softmax(pred_mask, dim=1)
        pred_mask = torch.argmax(pred_mask, dim=1)
        pred_mask = pred_mask.contiguous().view(-1)
        mask = mask.contiguous().view(-1)

        iou_per_class = []
        for clas in range(0, n_classes): #loop per pixel class
            true_class = pred_mask == clas
            true_label = mask == clas

            if true_label.long().sum().item() == 0: #no exist label in this loop
                iou_per_class.append(np.nan)
            else:
                intersect = torch.logical_and(true_class, true_label).sum().float().item()
                union = torch.logical_or(true_class, true_label).sum().float().item()

                iou = (intersect + smooth) / (union +smooth)
                iou_per_class.append(iou)
        return np.nanmean(iou_per_class)

In [ ]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def fit(epochs, model, train_loader, val_loader, criterion, optimizer, scheduler, patch=False):
    torch.cuda.empty_cache()
    train_losses = []
    test_losses = []
    val_iou = []; val_acc = []
    train_iou = []; train_acc = []
    lrs = []
    min_loss = np.inf
    decrease = 1 ; not_improve=0

    model.to(device)
    fit_time = time.time()
    for e in range(epochs):
        since = time.time()
        running_loss = 0
        iou_score = 0
        accuracy = 0
        #training loop
        model.train()
        for i, data in enumerate(tqdm(train_loader)):
            #training phase
            image_tiles, mask_tiles = data
            if patch:
                bs, n_tiles, c, h, w = image_tiles.size()

                image_tiles = image_tiles.view(-1,c, h, w)
                mask_tiles = mask_tiles.view(-1, h, w)
            
            image = image_tiles.to(device); mask = mask_tiles.to(device);
            #forward
            output = model(image)
            loss = criterion(output, mask)
            #evaluation metrics
            iou_score += mIoU(output, mask)
            accuracy += pixel_accuracy(output, mask)
            #backward
            loss.backward()
            optimizer.step() #update weight          
            optimizer.zero_grad() #reset gradient
            
            #step the learning rate
            lrs.append(get_lr(optimizer))
            scheduler.step() 
            
            running_loss += loss.item()
            
        else:
            model.eval()
            test_loss = 0
            test_accuracy = 0
            val_iou_score = 0
            #validation loop
            with torch.no_grad():
                for i, data in enumerate(tqdm(val_loader)):
                    #reshape to 9 patches from single image, delete batch size
                    image_tiles, mask_tiles = data

                    if patch:
                        bs, n_tiles, c, h, w = image_tiles.size()

                        image_tiles = image_tiles.view(-1,c, h, w)
                        mask_tiles = mask_tiles.view(-1, h, w)
                    
                    image = image_tiles.to(device); mask = mask_tiles.to(device);
                    output = model(image)
                    #evaluation metrics
                    val_iou_score +=  mIoU(output, mask)
                    test_accuracy += pixel_accuracy(output, mask)
                    #loss
                    loss = criterion(output, mask)                                  
                    test_loss += loss.item()
            
            #calculatio mean for each batch
            train_losses.append(running_loss/len(train_loader))
            test_losses.append(test_loss/len(val_loader))


            if min_loss > (test_loss/len(val_loader)):
                print('Loss Decreasing.. {:.3f} >> {:.3f} '.format(min_loss, (test_loss/len(val_loader))))
                min_loss = (test_loss/len(val_loader))
                decrease += 1
                if decrease % 5 == 0:
                    print('saving model...')
                    torch.save(model, 'Unet-Mobilenet_v2_mIoU-{:.3f}.pt'.format(val_iou_score/len(val_loader)))
                    

            if (test_loss/len(val_loader)) > min_loss:
                not_improve += 1
                min_loss = (test_loss/len(val_loader))
                print(f'Loss Not Decrease for {not_improve} time')
                if not_improve == 7:
                    print('Loss not decrease for 7 times, Stop Training')
                    break
            
            #iou
            val_iou.append(val_iou_score/len(val_loader))
            train_iou.append(iou_score/len(train_loader))
            train_acc.append(accuracy/len(train_loader))
            val_acc.append(test_accuracy/ len(val_loader))
            print("Epoch:{}/{}..".format(e+1, epochs),
                  "Train Loss: {:.3f}..".format(running_loss/len(train_loader)),
                  "Val Loss: {:.3f}..".format(test_loss/len(val_loader)),
                  "Train mIoU:{:.3f}..".format(iou_score/len(train_loader)),
                  "Val mIoU: {:.3f}..".format(val_iou_score/len(val_loader)),
                  "Train Acc:{:.3f}..".format(accuracy/len(train_loader)),
                  "Val Acc:{:.3f}..".format(test_accuracy/len(val_loader)),
                  "Time: {:.2f}m".format((time.time()-since)/60))
        
    history = {'train_loss' : train_losses, 'val_loss': test_losses,
               'train_miou' :train_iou, 'val_miou':val_iou,
               'train_acc' :train_acc, 'val_acc':val_acc,
               'lrs': lrs}
    print('Total time: {:.2f} m' .format((time.time()- fit_time)/60))
    return history

In [ ]:
max_lr = 1e-3
epoch = 25
weight_decay = 1e-4

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=max_lr, weight_decay=weight_decay)
sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epoch,
                                            steps_per_epoch=len(train_loader))

history = fit(epoch, model, train_loader, val_loader, criterion, optimizer, sched)

In [ ]:
torch.save(model, 'Unet-Resnet2_160m_25ep.pt')

In [ ]:
def plot_loss(history):
    plt.plot(history['val_loss'], label='val', marker='o')
    plt.plot( history['train_loss'], label='train', marker='o')
    plt.title('Loss per epoch'); plt.ylabel('loss');
    plt.xlabel('epoch')
    plt.legend(), plt.grid()
    plt.show()
    
def plot_score(history):
    plt.plot(history['train_miou'], label='train_mIoU', marker='*')
    plt.plot(history['val_miou'], label='val_mIoU',  marker='*')
    plt.title('Score per epoch'); plt.ylabel('mean IoU')
    plt.xlabel('epoch')
    plt.legend(), plt.grid()
    plt.show()
    
def plot_acc(history):
    plt.plot(history['train_acc'], label='train_accuracy', marker='*')
    plt.plot(history['val_acc'], label='val_accuracy',  marker='*')
    plt.title('Accuracy per epoch'); plt.ylabel('Accuracy')
    plt.xlabel('epoch')
    plt.legend(), plt.grid()
    plt.show()

In [ ]:
plot_loss(history)
plot_score(history)
plot_acc(history)

In [ ]:

def plot_loss(history, filename='loss.png'):
    plt.plot(history['val_loss'], label='val', marker='o')
    plt.plot(history['train_loss'], label='train', marker='o')
    plt.title('Loss per epoch')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend()
    plt.grid()
    plt.savefig(filename)
    plt.close()

def plot_score(history, filename='score.png'):
    plt.plot(history['train_miou'], label='train_mIoU', marker='*')
    plt.plot(history['val_miou'], label='val_mIoU', marker='*')
    plt.title('Score per epoch')
    plt.ylabel('mean IoU')
    plt.xlabel('epoch')
    plt.legend()
    plt.grid()
    plt.savefig(filename)
    plt.close()

def plot_acc(history, filename='accuracy.png'):
    plt.plot(history['train_acc'], label='train_accuracy', marker='*')
    plt.plot(history['val_acc'], label='val_accuracy', marker='*')
    plt.title('Accuracy per epoch')
    plt.ylabel('Accuracy')
    plt.xlabel('epoch')
    plt.legend()
    plt.grid()
    plt.savefig(filename)
    plt.close()

In [ ]:
plot_loss(history)
plot_score(history)
plot_acc(history)

### Analisi dei Risultati
Analizziamo i risultati del modello. Questa sezione include la valutazione delle metriche di prestazione, visualizzazioni dei risultati della segmentazione e un'analisi complessiva delle prestazioni del modello.


In [ ]:
class DriveTestDataset(Dataset):
    
    def __init__(self, img_path, mask_path, X, transform=None):
        self.img_path = img_path
        self.mask_path = mask_path
        self.X = X
        self.transform = transform
      
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        img = cv2.imread(self.img_path + self.X[idx] + '.jpg')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.mask_path + self.X[idx] + '.png', cv2.IMREAD_GRAYSCALE)
        
        if self.transform is not None:
            aug = self.transform(image=img, mask=mask)
            img = Image.fromarray(aug['image'])
            mask = aug['mask']
        
        if self.transform is None:
            img = Image.fromarray(img)
        
        mask = torch.from_numpy(mask).long()
        
        return img, mask


t_test = A.Resize(risoluz_vert, risoluz_orizz, interpolation=cv2.INTER_NEAREST)
test_set = DriveTestDataset(IMAGE_PATH, MASK_PATH, X_test, transform=t_test)

In [ ]:
class DriveTestDataset2(Dataset):
    def __init__(self, img_path, mask_path, X, mean, std, transform=None):
        self.img_path = img_path
        self.mask_path = mask_path
        self.X = X
        self.transform = transform
        self.mean = mean
        self.std = std
      
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        img = cv2.imread(self.img_path + self.X[idx] + '.jpg')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.mask_path + self.X[idx] + '.png', cv2.IMREAD_GRAYSCALE)
        
        if self.transform is not None:
            aug = self.transform(image=img, mask=mask)
            img = aug['image']
            mask = aug['mask']
        
        img = Image.fromarray(img)
        
        t = T.Compose([T.ToTensor(), T.Normalize(self.mean, self.std)])
        img = t(img)
        mask = torch.from_numpy(mask).long()
        
        return img, mask
    
# Caricamento del dataset di test
t_test = A.Compose([A.Resize(risoluz_vert, risoluz_orizz, interpolation=cv2.INTER_NEAREST)])
test_set2 = DriveTestDataset2(IMAGE_PATH, MASK_PATH, X_test, mean, std, transform=t_test)

In [ ]:
def predict_image_mask_miou(model, image, mask, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
    model.eval()
    t = T.Compose([T.ToTensor(), T.Normalize(mean, std)])
    image = t(image)
    model.to(device); image=image.to(device)
    mask = mask.to(device)
    with torch.no_grad():
        
        image = image.unsqueeze(0)
        mask = mask.unsqueeze(0)
        
        output = model(image)
        score = mIoU(output, mask)
        masked = torch.argmax(output, dim=1)
        masked = masked.cpu().squeeze(0)
    return masked, score

In [ ]:
def predict_image_mask_pixel(model, image, mask, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
    model.eval()
    t = T.Compose([T.ToTensor(), T.Normalize(mean, std)])
    image = t(image)
    model.to(device); image=image.to(device)
    mask = mask.to(device)
    with torch.no_grad():
        
        image = image.unsqueeze(0)
        mask = mask.unsqueeze(0)
        
        output = model(image)
        acc = pixel_accuracy(output, mask)
        masked = torch.argmax(output, dim=1)
        masked = masked.cpu().squeeze(0)
    return masked, acc

In [ ]:
def miou_score(model, test_set):
    score_iou = []
    for i in tqdm(range(len(test_set))):
        img, mask = test_set[i]
        pred_mask, score = predict_image_mask_miou(model, img, mask)
        score_iou.append(score)
    return score_iou

In [ ]:
mob_miou = miou_score(model, test_set)

In [ ]:
def pixel_acc(model, test_set):
    accuracy = []
    for i in tqdm(range(len(test_set))):
        img, mask = test_set[i]
        pred_mask, acc = predict_image_mask_pixel(model, img, mask)
        accuracy.append(acc)
    return accuracy

In [ ]:
mob_acc = pixel_acc(model, test_set)

In [ ]:
# Funzione per misurare i tempi di inferenza
def measure_inference_time(model, data_loader, device):
    model.eval()
    total_time = 0
    with torch.no_grad():
        for images, masks in tqdm(data_loader):
            images = images.to(device)
            start_time = time.time()
            _ = model(images)
            end_time = time.time()
            total_time += (end_time - start_time)
    avg_inference_time = total_time / len(data_loader.dataset)
    return total_time, avg_inference_time

In [ ]:
# Dataloader per il test set
test_loader = DataLoader(test_set2, batch_size=batch_size, shuffle=False)

# Misurazione dei tempi di inferenza
total_inference_time, avg_inference_time = measure_inference_time(model, test_loader, device)
print(f'Tempo medio di inferenza per immagine: {avg_inference_time:.4f} secondi')
print(f'Tempo totale di inferenza: {total_inference_time:.4f} secondi')

In [ ]:
print('Test Set mIoU', np.mean(mob_miou))
print('Test Set Pixel Accuracy', np.mean(mob_acc))

In [ ]:
def show_comparison_masks(test_set, model):
    indices = [0, 40, 200]  # Indici per la prima, quinta e ultima immagine
    fig, axes = plt.subplots(len(indices), 3, figsize=(20, len(indices) * 5))
    
    for i, idx in enumerate(indices):
        image, mask = test_set[idx]
        pred_mask, miou_score = predict_image_mask_miou(model, image, mask)
        pred_mask_pixel, pixel_accuracy = predict_image_mask_pixel(model, image, mask)
        
        axes[i, 0].imshow(image)
        axes[i, 0].set_title('Picture')
        axes[i, 0].set_axis_off()

        axes[i, 1].imshow(mask)
        axes[i, 1].set_title('Ground Truth')
        axes[i, 1].set_axis_off()

        axes[i, 2].imshow(pred_mask)
        axes[i, 2].set_title(f'{model_name} | mIoU {miou_score:.3f} | Pixel Accuracy {pixel_accuracy:.3f}')
        axes[i, 2].set_axis_off()

    plt.tight_layout()
    plt.show()

def save_comparison_masks(test_set, model, save_path='comparison_masks'):
    os.makedirs(save_path, exist_ok=True)
    indices = [0, 40, 200]  # Indici per la prima, quinta e ultima immagine
    
    for i, idx in enumerate(indices):
        image, mask = test_set[idx]
        pred_mask, miou_score = predict_image_mask_miou(model, image, mask)
        pred_mask_pixel, pixel_accuracy = predict_image_mask_pixel(model, image, mask)

        # Salva l'immagine originale
        plt.figure(figsize=(8, 4))
        plt.imshow(image)
        plt.title('Picture')
        plt.axis('off')
        plt.tight_layout(pad=1)  # Riduce il padding attorno all'immagine
        plt.savefig(f'{save_path}/{model_name}_image_{i+1}.png', bbox_inches='tight', pad_inches=0.1)
        plt.close()

        # Salva il Ground Truth
        plt.figure(figsize=(8, 4))
        plt.imshow(mask)
        plt.title('Ground Truth')
        plt.axis('off')
        plt.tight_layout(pad=1)  # Riduce il padding attorno all'immagine
        plt.savefig(f'{save_path}/{model_name}_ground_truth_{i+1}.png', bbox_inches='tight', pad_inches=0.1)
        plt.close()

        # Salva la maschera predetta
        plt.figure(figsize=(8, 4))
        plt.imshow(pred_mask)
        plt.title(f'{model_name} | mIoU {miou_score:.3f} | Pixel Accuracy {pixel_accuracy:.3f}')
        plt.axis('off')
        plt.tight_layout(pad=1)  # Riduce il padding attorno all'immagine
        plt.savefig(f'{save_path}/{model_name}_pred_mask_{i+1}.png', bbox_inches='tight', pad_inches=0.1)
        plt.close()
        
def save_predicted_masks_with_metrics(test_set, model, mod, save_path='predicted_masks'):
    os.makedirs(save_path, exist_ok=True)
    indices = [0, 40, 200]  # Indici per la prima, quinta e ultima immagine
    if mod == "quantized":
        modifica = "Quantized"
    elif mod == "pruned":
        modifica = "Pruned"
    else:
        modifica = "Quantized & Pruned"
    
    for i, idx in enumerate(indices):
        image, mask = test_set[idx]
        pred_mask, miou_score = predict_image_mask_miou(model, image, mask)
        pred_mask_pixel, pixel_accuracy = predict_image_mask_pixel(model, image, mask)

        plt.figure(figsize=(8, 4))
        plt.imshow(pred_mask)
        plt.title(f'{modifica} {model_name} | mIoU {miou_score:.3f} | Pixel Accuracy {pixel_accuracy:.3f}')
        plt.axis('off')

        plt.tight_layout(pad=1)  # Riduce il padding attorno all'immagine
        plt.savefig(f'{save_path}/{model_name}_predicted_mask_{mod}_{i+1}.png', bbox_inches='tight', pad_inches=0.1)
        plt.close()


In [ ]:
show_comparison_masks(test_set, model)

In [ ]:
save_comparison_masks(test_set, model)

In [ ]:
"""
import torch
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

def evaluate_segmentation(model, dataloader, num_classes):
    model.eval()
    precisions = np.zeros(num_classes)
    recalls = np.zeros(num_classes)
    f1s = np.zeros(num_classes)
    n_samples = 0
    
    with torch.no_grad():
        for images, true_masks in tqdm(dataloader, desc="Evaluating"):
            images = images.to(device)
            true_masks = true_masks.to(device)

            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)

            preds = preds.cpu().numpy().flatten()
            true_masks = true_masks.cpu().numpy().flatten()

            for cls in range(num_classes):
                cls_mask = (true_masks == cls)
                cls_pred = (preds == cls)
                
                precision = precision_score(cls_mask, cls_pred, zero_division=0)
                recall = recall_score(cls_mask, cls_pred, zero_division=0)
                f1 = f1_score(cls_mask, cls_pred, zero_division=0)

                precisions[cls] += precision
                recalls[cls] += recall
                f1s[cls] += f1

            n_samples += 1

    precisions /= n_samples
    recalls /= n_samples
    f1s /= n_samples

    for cls in range(num_classes):
        print(f'Class {cls}: Precision = {precisions[cls]:.4f}, Recall = {recalls[cls]:.4f}, F1-score = {f1s[cls]:.4f}')

    return precisions, recalls, f1s

# Esempio di utilizzo
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)
# num_classes = 10  # Adatta questo numero alle tue classi
precisions, recalls, f1s = evaluate_segmentation(model, test_loader, n_classes)
"""

### Quantizzazione e Pruning
Questa parte copre le tecniche di quantizzazione e pruning utilizzate per ottimizzare il modello, rendendolo più efficiente durante l'inferenza.


In [ ]:
import torchvision.models as models

# Funzione per quantizzare un layer lineare o convoluzionale con quantizzazione non lineare
def nonlinear_quantize_layer(layer, num_bits):
    quantized_layer = layer
    if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
        # Calcola i parametri di quantizzazione non lineare
        abs_max = torch.abs(layer.weight).max()
        scale = abs_max / (2 ** (num_bits - 1) - 1)
        quantized_weight = torch.clamp(layer.weight, -abs_max.item(), abs_max.item())
        quantized_weight = torch.round(quantized_weight / scale)
        quantized_weight = torch.clamp(quantized_weight, -2**(num_bits-1), 2**(num_bits-1) - 1)

        quantized_layer.weight.data = quantized_weight
        if layer.bias is not None:
            quantized_layer.bias.data = torch.round(layer.bias.data / scale)
    return quantized_layer

# Funzione per quantizzare un intero modello con quantizzazione non lineare
def nonlinear_quantize_model(model, num_bits):
    quantized_model = model
    for name, layer in quantized_model.named_children():
        if isinstance(layer, nn.Sequential) or isinstance(layer, nn.ModuleList):
            quantized_model.add_module(name, nonlinear_quantize_model(layer, num_bits))
        elif isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
            quantized_model.add_module(name, nonlinear_quantize_layer(layer, num_bits))
        else:
            quantized_model.add_module(name, layer)
    return quantized_model


In [ ]:
# Quantizza il modello di segmentazione semantica
model_quantized = copy.deepcopy(model)
model_quantized = nonlinear_quantize_model(model_quantized, num_bits=8)

In [ ]:
"""
def print_size_of_model(model, label=""):
    torch.save(model.state_dict(), "temp.p")
    size=os.path.getsize("temp.p")
    print("model: ",label,' \t','Size (KB):', size/1e3)
    os.remove('temp.p')
    return size

# compare the sizes
f=print_size_of_model(model,"Original")
q=print_size_of_model(model_quantized,"Quantized")
if f > q:
    print("{0:.2f} times smaller".format(f/q))
elif f < q:
    print("{0:.2f} times larger".format(q/f))
else:
    print("Same size")
"""

In [ ]:
"""
for name, param in model_quantized.named_parameters():
    print(name, param.dtype)
"""

In [ ]:
import torch.nn.utils.prune as prune
# Funzione per applicare il pruning strutturato
def structured_prune_model(model, amount):
    for module_name, module in model.named_modules():
        if isinstance(module, torch.nn.Conv2d):
            prune.ln_structured(module, name='weight', amount=amount, n=1, dim=0)
        elif isinstance(module, torch.nn.Linear):
            prune.ln_structured(module, name='weight', amount=amount, n=1, dim=1)
    return model

In [ ]:
# Applicazione del pruning strutturato
model_pruned = copy.deepcopy(model)
model_quantized_pruned = copy.deepcopy(model_quantized)

model_pruned = structured_prune_model(model_pruned, amount=0.03)
model_quantized_pruned = structured_prune_model(model_quantized_pruned, amount=0.03)

### Analisi dei Risultati dopo Quantizzazione
Analizziamo i risultati del modello dopo la quantizzazione. Questa sezione include la valutazione delle metriche di prestazione, visualizzazioni dei risultati della segmentazione e un'analisi complessiva delle prestazioni del modello.


In [ ]:
mob_miou_quantized = miou_score(model_quantized, test_set)

In [ ]:
mob_acc_quantized = pixel_acc(model_quantized, test_set)

In [ ]:
print('Test Set mIoU', np.mean(mob_miou_quantized))
print('Test Set Pixel Accuracy', np.mean(mob_acc_quantized))

In [ ]:
# Misurazione dei tempi di inferenza
total_inference_time, avg_inference_time = measure_inference_time(model_quantized, test_loader, device)
print(f'Tempo medio di inferenza per immagine: {avg_inference_time:.4f} secondi')
print(f'Tempo totale di inferenza: {total_inference_time:.4f} secondi')

In [ ]:
mod = "quantized"
save_predicted_masks_with_metrics(test_set, model_quantized, mod)
show_comparison_masks(test_set, model_quantized)

### Analisi dei Risultati dopo Pruning
Analizziamo i risultati del modello dopo il pruning. Questa sezione include la valutazione delle metriche di prestazione, visualizzazioni dei risultati della segmentazione e un'analisi complessiva delle prestazioni del modello.


In [ ]:
mob_miou_pruned = miou_score(model_pruned, test_set)

In [ ]:
mob_acc_pruned = pixel_acc(model_pruned, test_set)

In [ ]:
print('Test Set mIoU', np.mean(mob_miou_pruned))
print('Test Set Pixel Accuracy', np.mean(mob_acc_pruned))

In [ ]:
# Misurazione dei tempi di inferenza
total_inference_time, avg_inference_time = measure_inference_time(model_pruned, test_loader, device)
print(f'Tempo medio di inferenza per immagine: {avg_inference_time:.4f} secondi')
print(f'Tempo totale di inferenza: {total_inference_time:.4f} secondi')

In [ ]:
mod = "pruned"
save_predicted_masks_with_metrics(test_set, model_pruned, mod)
show_comparison_masks(test_set, model_pruned)

### Analisi dei Risultati dopo Quantizzazione e Pruning
Infine, analizziamo i risultati del modello dopo quantizzazione e pruning. Questa sezione include la valutazione delle metriche di prestazione, visualizzazioni dei risultati della segmentazione e un'analisi complessiva delle prestazioni del modello.

In [ ]:
mob_miou_quantized_pruned = miou_score(model_quantized_pruned, test_set)

In [ ]:
mob_acc_quantized_pruned = pixel_acc(model_quantized_pruned, test_set)

In [ ]:
print('Test Set mIoU', np.mean(mob_miou_quantized_pruned))
print('Test Set Pixel Accuracy', np.mean(mob_acc_quantized_pruned))

In [ ]:
# Misurazione dei tempi di inferenza
total_inference_time, avg_inference_time = measure_inference_time(model_quantized_pruned, test_loader, device)
print(f'Tempo medio di inferenza per immagine: {avg_inference_time:.4f} secondi')
print(f'Tempo totale di inferenza: {total_inference_time:.4f} secondi')

In [ ]:
mod = "quantized_pruned"
save_predicted_masks_with_metrics(test_set, model_quantized_pruned, mod)
show_comparison_masks(test_set, model_quantized_pruned)